In [1]:
import pandas as pd 
import numpy as np 
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Install Driver 
driver = webdriver.Chrome(ChromeDriverManager().install())

# Setting up urls &page=2
search_url = ["https://www.price.com.hk/ec-product-list.php?list_id=1218", "https://www.price.com.hk/ec-product-list.php?list_id=1218&page=2"]


In [4]:
from collections import defaultdict

product_info = defaultdict(list)
product_list = []
price_list = []

for url in search_url:
    driver.get(url)
    product_prices_raw = driver.find_elements(By.CLASS_NAME, "text-price-number")
    product_name_raw = driver.find_elements(By.CLASS_NAME, "ec-product-title")

    # product name 
    for product in product_name_raw:
        product_list.append(product.text)

    # product price - skipping the discounted prices 
    for index, price in enumerate(product_prices_raw, start = 1):
        if index % 2 == 1:
            continue
        price_list.append(price.text)
    product_info['product_name'].extend(product_list)
    product_info['product_price'].extend(price_list)

# Collborative Filtering 

In [5]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_instance = TfidfVectorizer()
X = tfidf_instance.fit_transform(product_info.get('product_name'))

from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=5).fit(X)

In [67]:
user_input = 5
_index = nbrs.kneighbors(X[user_input], return_distance = False)[0].tolist()
_index.remove(user_input)
stuffs_to_recommend = []
print(f"You have recently purchased: {product_info.get('product_name')[user_input]}")
for i in (_index): 
    stuffs_to_recommend.append(product_info.get('product_name')[i])
stuffs_to_recommend

You have recently purchased: 3M 濾水系統濾芯 [AP2-C405-G]


['3M 濾水系統濾芯 [AP2-C405-G]',
 'ITSU 體脂磅 [2色]',
 'ITSU 體脂磅 [2色]',
 'MSI G244F 24" 170Hz 電競顯示器',
 'Razer DeathAdder V2 Pro 專業版無線遊戲滑鼠',
 'FujiFilm 富士 Instax Mini 9 即影即有相機 [鈷藍]',
 'Braun 百靈 Series 5 乾濕兩用電鬚刨 [51-B1200s]',
 'Microsoft Office 365 [個人電子下載版/家庭版盒裝]',
 'EGO Exinno 300W 即時輸出顯示USB充電器']